<a href="https://colab.research.google.com/github/bhargavyagnik/text_generation/blob/main/text_generation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Text Generation using Transformers library

This work is based on the Huggingface tutorial titled "How to generate text: using different decoding methods for language generation with Transformers" by  [Patrick Von Platen](https://huggingface.co/patrickvonplaten). 

I highly recommend you to check that ! It is quite detailed and very well explained

In [1]:
!pip install -q git+https://github.com/huggingface/transformers.git
!pip install -q tensorflow==2.1

  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done
     |████████████████████████████████| 890kB 5.3MB/s 
     |████████████████████████████████| 2.9MB 17.8MB/s 
     |████████████████████████████████| 421.8MB 37kB/s 
     |████████████████████████████████| 51kB 5.0MB/s 
     |████████████████████████████████| 3.9MB 49.4MB/s 
     |████████████████████████████████| 450kB 48.1MB/s 
ERROR: tensorflow-probability 0.11.0 has requirement gast>=0.3.2, but you'll have gast 0.2.2 which is incompatible.
